In [1]:
%reload_ext autoreload
%autoreload 2

import logging
import sys
from itertools import product
from pathlib import Path


# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

In [2]:
from pd3f import extract
from tqdm import tqdm
from joblib import Parallel, delayed

In [ ]:
! rm -rf ~/.cache/pd3f/dehyphen

In [ ]:
data_folder = '/mnt/data/ptf_ddd/bmjv/proc_pdf/'
version = '0.3.0'
results_folder = f"/mnt/data/ptf_ddd/bmjv/results/{version}/"

In [ ]:
def work_on_one_folder(folder_name, experimental, fast, tables=False, force_gpu=True):
    def to_work(f):
        try:
#         if True:
            text, _ = extract(str(f), lang="multi-v0-fast", experimental=experimental, tables=tables, force_gpu=force_gpu, fast=fast)
            enc = f"/ex_{experimental}/fa_{fast}/"
            Path(f'{results_folder}{folder_name}{enc}').mkdir(exist_ok=True, parents=True)

            Path(f'{results_folder}{folder_name}{enc}{f.name}').with_suffix('.txt').write_text(text)
        except Exception as e:
            return f"error {folder_name} {f} {e} "
        except AssertionError as e:
            return f"error {folder_name} {f} {e} "

    return Parallel(n_jobs=8)(delayed(to_work)(f) for f in tqdm(list(Path(f'{data_folder}{folder_name}/').glob("*.pdf"))[:]))

In [ ]:
raise_error = False
all_errors = ""
for x in product(['digital_en', 'digital_de', 'analog_en', 'analog_de'], [True, False], [True, False]):
    print(*x)
    errs = work_on_one_folder(*x)
    errs = [x for x in errs if x is not None]
    if len(errs) > 0:
        print(errs)
        if raise_error:
            raise ValueError

        all_errors += str(errs) + '\n'



In [7]:
Path(results_folder + 'log.txt').write_text(all_errors)

0